In [1]:
import json, requests, re
import numpy as np
from pprint import pprint
from watson_developer_cloud import DiscoveryV1

In [2]:
def getIBMCreds():
    credentials = json.load(open('../credentials.json'))
    ibmCreds = credentials['ibm']['discovery']
    return ibmCreds

In [3]:
def getQuery():
    query = input('Enter the query to be searched: ')
    return(query)

In [4]:
def constructQuery(query):
    query_options = {'natural_language_query': query,
                 'count' : 3,
                 'passages':True}
    return(query_options)

In [5]:
def getQueryResults(discoveryDetails, queryOptions):
    env_id, collections, discovery = discoveryDetails
    queryResults = discovery.query(environment_id=env_id, 
                               collection_id=collections[0]['collection_id'],
                               natural_language_query=queryOptions['natural_language_query'], 
                               passages=queryOptions['passages'], count=queryOptions['count'])
    return(queryResults)

In [38]:
def cleanPassage(passage):
    tagRegEx = re.compile('<.*?>')
    cleanedText = re.sub(tagRegEx, '', passage)
    cleanedText = cleanedText.rstrip()
    cleanedText = cleanedText.replace('\n', '. ')
    return(cleanedText)

In [39]:
def getPassage(queryResults):
    passages = []
    for passage in queryResults['passages']:
        passages.append(passage)
    passageScore = 0
    targetPassages = []
    for passage in passages:
        for key in passage.keys():
            if(passageScore < passage['passage_score']):
                passageScore = passage['passage_score']
                targetPassages.append(passage)
    targetPassage = targetPassages[0]['passage_text']
    cleanedPassage = cleanPassage(targetPassage)
    print("Target passage is: ".format(" "))
    print(targetPassage)
    return(cleanedPassage)

In [40]:
def askDiscovery():
    ibmCreds = getIBMCreds()
    discovery = DiscoveryV1(version='2017-11-07',
                            username=ibmCreds['username'],
                            password=ibmCreds['password'])
    env = discovery.list_environments()['environments'][1]
    env_id = env['environment_id']
    collections = discovery.list_collections(env_id)
    collections = collections['collections']
    discoveryDetails = (env_id, collections, discovery)
    query = getQuery()
    print("")
    queryOptions = constructQuery(query)
    queryResults = getQueryResults(discoveryDetails, queryOptions)
    passage = getPassage(queryResults)
    print("Passage to be returned is: {}".format(""))
    print("========================================================================================")
    print(passage)
    print("========================================================================================")
    return(passage)

In [46]:
a = askDiscovery()

Enter the query to be searched: what is Perception

Target passage is: 
Functions

Information from the sense organs is collected in the brain. There it is used to determine what actions the organism is to take. The brain processes the raw data to extract information about the structure of the environment.

Perception

The human brain is provided
Passage to be returned is: 
Functions. . Information from the sense organs is collected in the brain. There it is used to determine what actions the organism is to take. The brain processes the raw data to extract information about the structure of the environment.. . Perception. . The human brain is provided


In [6]:
# configurations = discovery.list_configurations(
#     environment_id=env_id)
# configurations = configurations['configurations']